In [1]:
import pandas as pd

data = pd.read_csv("/home/fm-pc-lt-228/Desktop/covid_nepali_tweets/covid19_tweeter_dataset (copy).csv")

In [2]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

In [3]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
#     import pdb;pdb.set_trace()
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [4]:
data['word_embeddings'] = data['Tweet'].apply(get_bert_embedding_sentence)

In [5]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [6]:
X,y = data['word_embeddings'], data['Label']

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [8]:
svc = SVC(probability=True)

In [9]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [10]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [11]:
print(confusion_matrix(test_y, svc_pred))

[[2286   65  470]
 [ 286  534  479]
 [ 506   68 2386]]


In [12]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

          -1       0.74      0.81      0.78      2821
           0       0.80      0.41      0.54      1299
           1       0.72      0.81      0.76      2960

    accuracy                           0.74      7080
   macro avg       0.75      0.68      0.69      7080
weighted avg       0.74      0.74      0.73      7080



In [13]:
accuracy_score(test_y, svc_pred)

0.7353107344632769

In [15]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [19]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, random_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.5796610169491525
[[1377    0 1444]
 [ 135   20 1144]
 [ 253    0 2707]]
              precision    recall  f1-score   support

          -1       0.78      0.49      0.60      2821
           0       1.00      0.02      0.03      1299
           1       0.51      0.91      0.66      2960

    accuracy                           0.58      7080
   macro avg       0.76      0.47      0.43      7080
weighted avg       0.71      0.58      0.52      7080



In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(train_X.tolist(), train_y)



NameError: name 'y_test' is not defined

In [20]:
gbc_pred = gb_clf.predict(test_X.tolist())
print(accuracy_score(test_y, gbc_pred))

print(confusion_matrix(test_y, gbc_pred))

print(classification_report(test_y, gbc_pred))

0.6929378531073447
[[2087  145  589]
 [ 296  555  448]
 [ 535  161 2264]]
              precision    recall  f1-score   support

          -1       0.72      0.74      0.73      2821
           0       0.64      0.43      0.51      1299
           1       0.69      0.76      0.72      2960

    accuracy                           0.69      7080
   macro avg       0.68      0.64      0.65      7080
weighted avg       0.69      0.69      0.69      7080



# Naive Bayes

In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [22]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.6152542372881356
[[1618  462  741]
 [ 209  668  422]
 [ 451  439 2070]]
              precision    recall  f1-score   support

          -1       0.71      0.57      0.63      2821
           0       0.43      0.51      0.47      1299
           1       0.64      0.70      0.67      2960

    accuracy                           0.62      7080
   macro avg       0.59      0.60      0.59      7080
weighted avg       0.63      0.62      0.62      7080



# Ada Boost

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [24]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.6676553672316384
[[2033  155  633]
 [ 304  528  467]
 [ 610  184 2166]]
              precision    recall  f1-score   support

          -1       0.69      0.72      0.70      2821
           0       0.61      0.41      0.49      1299
           1       0.66      0.73      0.70      2960

    accuracy                           0.67      7080
   macro avg       0.65      0.62      0.63      7080
weighted avg       0.66      0.67      0.66      7080



# Decision Tree

In [25]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [26]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.5269774011299435
[[ 957    0 1864]
 [  81    0 1218]
 [ 186    0 2774]]
              precision    recall  f1-score   support

          -1       0.78      0.34      0.47      2821
           0       0.00      0.00      0.00      1299
           1       0.47      0.94      0.63      2960

    accuracy                           0.53      7080
   macro avg       0.42      0.43      0.37      7080
weighted avg       0.51      0.53      0.45      7080



/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

# Extra Trees Classifier

In [27]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [28]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.701271186440678
[[1990   47  784]
 [ 250  472  577]
 [ 412   45 2503]]
              precision    recall  f1-score   support

          -1       0.75      0.71      0.73      2821
           0       0.84      0.36      0.51      1299
           1       0.65      0.85      0.73      2960

    accuracy                           0.70      7080
   macro avg       0.75      0.64      0.66      7080
weighted avg       0.72      0.70      0.69      7080



# Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000.0)

In [30]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.7182203389830508
[[2154  151  516]
 [ 272  612  415]
 [ 468  173 2319]]
              precision    recall  f1-score   support

          -1       0.74      0.76      0.75      2821
           0       0.65      0.47      0.55      1299
           1       0.71      0.78      0.75      2960

    accuracy                           0.72      7080
   macro avg       0.70      0.67      0.68      7080
weighted avg       0.71      0.72      0.71      7080



# KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [32]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.6841807909604519
[[2132  129  560]
 [ 397  537  365]
 [ 640  145 2175]]
              precision    recall  f1-score   support

          -1       0.67      0.76      0.71      2821
           0       0.66      0.41      0.51      1299
           1       0.70      0.73      0.72      2960

    accuracy                           0.68      7080
   macro avg       0.68      0.63      0.65      7080
weighted avg       0.68      0.68      0.68      7080



# MLP

In [33]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [34]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.7046610169491525
[[2086  210  525]
 [ 254  705  340]
 [ 527  235 2198]]
              precision    recall  f1-score   support

          -1       0.73      0.74      0.73      2821
           0       0.61      0.54      0.58      1299
           1       0.72      0.74      0.73      2960

    accuracy                           0.70      7080
   macro avg       0.69      0.67      0.68      7080
weighted avg       0.70      0.70      0.70      7080

